In [1]:
import glob
import pandas as pd
catfn = glob.glob("train/cat.*.jpg")
dogfn = glob.glob("train/dog.*.jpg")
catans = [0] * len(catfn)
dogans = [1] * len(dogfn)
pd.DataFrame({
    "path":catfn + dogfn,
    "ans"
})

['train/cat.5077.jpg',
 'train/cat.2718.jpg',
 'train/cat.10151.jpg',
 'train/cat.3406.jpg',
 'train/cat.4369.jpg',
 'train/cat.7660.jpg',
 'train/cat.8553.jpg',
 'train/cat.9895.jpg',
 'train/cat.1211.jpg',
 'train/cat.6218.jpg',
 'train/cat.1577.jpg',
 'train/cat.12020.jpg',
 'train/cat.7106.jpg',
 'train/cat.8235.jpg',
 'train/cat.952.jpg',
 'train/cat.3360.jpg',
 'train/cat.11529.jpg',
 'train/cat.10637.jpg',
 'train/cat.5711.jpg',
 'train/cat.946.jpg',
 'train/cat.3374.jpg',
 'train/cat.10623.jpg',
 'train/cat.5705.jpg',
 'train/cat.1563.jpg',
 'train/cat.12034.jpg',
 'train/cat.7112.jpg',
 'train/cat.8221.jpg',
 'train/cat.7674.jpg',
 'train/cat.8547.jpg',
 'train/cat.9881.jpg',
 'train/cat.1205.jpg',
 'train/cat.9659.jpg',
 'train/cat.5063.jpg',
 'train/cat.10145.jpg',
 'train/cat.3412.jpg',
 'train/cat.7884.jpg',
 'train/cat.9671.jpg',
 'train/cat.6542.jpg',
 'train/cat.2724.jpg',
 'train/cat.11273.jpg',
 'train/cat.4355.jpg',
 'train/cat.4433.jpg',
 'train/cat.11515.jpg',
 'tr